# Load the Model

In [52]:
import pandas as pd
import nltk
import re, string
import numpy as np
import pickle

from tensorflow.keras.models import load_model
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model

In [53]:
tweet = "@ShreyaBafna3 Now you idiots claim that people who tried to stop him from becoming a terrorist made him a terrorist. Islamically brain dead."

In [54]:
# Load the model

model = load_model("../Model/CyberBully_DL_Model.h5")
test_data = pd.read_csv("../Dataset/twitter_parsed_dataset.csv").sample(5)

In [55]:
# turn a doc into clean tokens
def clean_doc(doc, vocab):
    # split into tokens by white space
    tokens = doc.split()
    # prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    tokens = [re_punc.sub('', w) for w in tokens]
    # filter out tokens not in vocab
    tokens = [w for w in tokens if w in vocab]
    tokens = ' '.join(tokens)
    return tokens

In [56]:
# integer encode and pad documents
def encode_docs(tokenizer, max_length, docs):
    # integer encode
    encoded = tokenizer.texts_to_sequences(docs)
    # pad sequences
    padded = pad_sequences(encoded, maxlen=max_length, padding='post')
    return padded

In [57]:
# classify a review as negative or positive
def predict_sentiment(review, vocab, tokenizer, max_length, model):
    # clean review
    line = clean_doc(review, vocab)
    # encode and pad review
    padded = encode_docs(tokenizer, max_length, [line])
    # predict sentiment
    yhat = model.predict(padded, verbose=0)
    # retrieve predicted percentage and label
    percent_pos = yhat[0,0]
    if round(percent_pos) == 0:
        return (1-percent_pos), 'NEGATIVE'
    return percent_pos, 'POSITIVE'

In [58]:
def read__vocab_list(filename):
    # open file
    file = open(filename, 'r', encoding="utf-8")
    # Read text
    vocab = file.read()
    # close file
    file.close()
    return vocab

In [59]:
vocab = read__vocab_list("../Model/vocab.txt")
vocab = vocab.split("\n")

In [60]:
Tokenizer = pickle.load(open("../Model/token.pickle","rb"))

In [61]:
result = predict_sentiment(tweet,vocab,Tokenizer,256,model)

In [62]:
result

(0.8250032514333725, 'NEGATIVE')